<a href="https://colab.research.google.com/github/Sristi222/Tic-Tac-Toe/blob/main/Try_for_report_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pytesseract

In [ ]:
import pytesseract

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
import pytesseract
import cv2
import numpy as np
from PIL import Image
import os
import re


In [ ]:
folder_path = '/content/drive/MyDrive/Colab Notebooks/Indian Currency Dataset'
pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract'


In [ ]:
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [ ]:
# Load directories
train_dir = os.path.join(folder_path, 'train')
validation_dir = os.path.join(folder_path, 'validation')
test_dir = os.path.join(folder_path, 'test')


In [ ]:
# Generate batches of augmented data
batch_size = 32
target_size = (224, 224)

train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=True
)

validation_generator = datagen.flow_from_directory(
    validation_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False
)

test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False
)

Found 150 images belonging to 2 classes.
Found 62 images belonging to 2 classes.
Found 107 images belonging to 2 classes.


In [ ]:
# Load MobileNetV2 (Transfer Learning)
base_model = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3), include_top=False)
base_model.trainable = False  # Freeze base model


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
# Add custom layers on top
flatten_layer = tf.keras.layers.Flatten()(base_model.output)
dropout_layer = tf.keras.layers.Dropout(0.5)(flatten_layer)
output_layer = tf.keras.layers.Dense(1, activation='sigmoid')(dropout_layer)
model = tf.keras.models.Model(base_model.input, output_layer)


In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
# Train the model
num_epochs = 10
model.fit(train_generator, epochs=num_epochs, validation_data=validation_generator)


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


5/5 ━━━━━━━━━━━━━━━━━━━━ 86s 9s/step - accuracy: 0.4848 - loss: 4.1866 - val_accuracy: 0.5161 - val_loss: 3.7711
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 27s 4s/step - accuracy: 0.7059 - loss: 2.7242 - val_accuracy: 0.6774 - val_loss: 1.6726
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 19s 3s/step - accuracy: 0.6854 - loss: 1.9030 - val_accuracy: 0.8387 - val_loss: 0.7791
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 45s 6s/step - accuracy: 0.8673 - loss: 0.9006 - val_accuracy: 0.8871 - val_loss: 1.1213
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 64s 4s/step - accuracy: 0.8139 - loss: 1.0720 - val_accuracy: 0.8548 - val_loss: 0.5039
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 25s 4s/step - accuracy: 0.8043 - loss: 1.4198 - val_accuracy: 0.8871 - val_loss: 0.6943
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 32s 4s/step - accuracy: 0.8402 - loss: 0.9187 - val_accuracy: 0.8387 - val_loss: 0.6201
Epoch 8/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 27s 4s/step - accuracy: 0.8961 - loss: 0.3658 - val_accuracy: 0.8387 - val_loss: 0.5785
Epoch 9/10
5/5 ━━━━

In [ ]:
#Extract features for SVM
def extract_features(generator, model):
    features = []
    labels = []
    for batch_images, batch_labels in generator:
        batch_features = model.predict(batch_images)
        features.extend(batch_features)
        labels.extend(batch_labels)
        if len(features) >= generator.samples:  # Limit to dataset size
            break
    return np.array(features), np.array(labels)


In [ ]:
# Extract features from train and test sets
train_features, train_labels = extract_features(train_generator, tf.keras.models.Model(base_model.input, flatten_layer))
test_features, test_labels = extract_features(test_generator, tf.keras.models.Model(base_model.input, flatten_layer))


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


In [ ]:
# Normalize features
scaler = StandardScaler()
train_features = scaler.fit_transform(train_features)
test_features = scaler.transform(test_features)


In [ ]:
# Train an SVM classifier
svm_classifier = SVC(kernel='linear', probability=True)
svm_classifier.fit(train_features, train_labels)

# Evaluate SVM
svm_predictions = svm_classifier.predict(test_features)
print(classification_report(test_labels, svm_predictions))



SVC(kernel='linear', probability=True)

In [ ]:

# OCR for text validation
def extract_text_from_image(image_path):
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    text = pytesseract.image_to_string(gray, lang='eng+nep')  # Supports English and Nepali
    return text

def validate_currency_with_ocr(image_path):
    extracted_text = extract_text_from_image(image_path)
    if "\u0928\u0947\u092a\u093e\u0932 \u0930\u093e\u0937\u094d\u091f\u094d\u0930 \u092c\u0948\u0902\u0915" in extracted_text:  # Check for central bank text
        return "Text Validation Passed"
    else:
        return "Text Validation Failed"

In [ ]:

# Additional validations
def detect_hologram(image_path):
    img = cv2.imread(image_path)
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    bright_areas = cv2.inRange(hsv, (0, 0, 200), (255, 255, 255))  # Threshold for bright areas
    if cv2.countNonZero(bright_areas) > 1000:  # Check if hologram area is significant
        return "Hologram Detected"
    return "Hologram Missing"

def validate_color_pattern(image_path, template_path):
    img = cv2.imread(image_path)
    template = cv2.imread(template_path)
    hist_img = cv2.calcHist([img], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
    hist_template = cv2.calcHist([template], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
    correlation = cv2.compareHist(hist_img, hist_template, cv2.HISTCMP_CORREL)
    if correlation > 0.9:  # High correlation indicates match
        return "Color Pattern Validated"
    return "Color Pattern Mismatch"

def detect_watermark(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    edges = cv2.Canny(img, 100, 200)  # Detect edges
    if cv2.countNonZero(edges) > 1500:  # Edge count threshold
        return "Watermark Detected"
    return "Watermark Missing"

def detect_latent_image(image_path):
    img = cv2.imread(image_path)
    gradient = cv2.Laplacian(img, cv2.CV_64F)  # Detect gradient intensity
    if np.mean(gradient) > 50:  # Threshold for latent features
        return "Latent Image Detected"
    return "Latent Image Missing"

In [ ]:
# Comprehensive prediction function
def predict_currency(image_path):
    # Preprocess image for CNN
    img = Image.open(image_path).resize(target_size)
    img_array = np.array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)



In [ ]:
 # CNN prediction
cnn_prediction = model.predict(img_array)[0][0]
cnn_label = "Real" if cnn_prediction >= 0.5 else "Fake"



NameError: name 'img_array' is not defined